In [67]:
from pathlib import Path
import whisper
from tqdm import tqdm
from pprint import pprint

def get_output_filename(source_filename, main_data_path=data_path):
    # different naminf pre and post 2018:
    # Get first subfolder after '/Users/vigji/Desktop/sando-data/audio' parent:
    year = int(source_filename.relative_to(main_data_path).parts[0])
    if year < 2018:
        word_list_candidates_for_date = source_filename.parent.name.split(" ")
        # keep only numbers in the date:
        datelist = [x for x in word_list_candidates_for_date if x.isdigit()]
    else:
        word_list_candidates_for_date = source_filename.stem.split(" ")
        # keep only numbers in the date:
        datelist = [x for x in word_list_candidates_for_date if x.isdigit()]
        # print(word_list_candidates_for_date)

    date = "-".join(datelist)
    return f"{date}_{source_filename.stem.replace(' ', '-')}_whispered.txt"



data_path = Path("/Users/vigji/Desktop/sando-data/audio")

# find all files called *omelia*.mp3 in data_path subfolders until 2017:
all_omelie = list(data_path.rglob("*melia*.mp3"))

# add all mp3 files from 2018 onwards:
all_omelie += list([f for f in data_path.rglob("*.mp3") if any([str(year) in str(f) for year in range(2018, 2025)])])

all_omelie = sorted(all_omelie)

print("F")
pprint({f: get_output_filename(f) for f in all_omelie})

model = whisper.load_model("large-v3")

for omelia in tqdm(all_omelie):
    filename = omelia.stem + "_whispered.txt"
    if Path(filename).exists():
        continue

    result = model.transcribe(str(omelia), initial_prompt="Un'omelia registrata durante una messa.") 
    full_text = [t["text"] for t in result["segments"]]

    with open(omelia.parent / filename, "w") as f:
        f.write("\n".join(full_text)) 

736
{PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 01/2013 01 06/5 Omelia.mp3'): '2013-01-06_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 01/2013 01 20/5 Omelia.mp3'): '2013-01-20_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 02/2013 02 03/5 Omelia.mp3'): '2013-02-03_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 02/2013 02 10/5 Omelia.mp3'): '2013-02-10_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 02/2013 02 13/5 Omelia.mp3'): '2013-02-13_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 02/2013 02 17/5 Omelia.mp3'): '2013-02-17_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 02/2013 02 24/5 Omelia.mp3'): '2013-02-24_5-Omelia_whispered.txt',
 PosixPath('/Users/vigji/Desktop/sando-data/audio/2013/2013 06/2013 06 02 comunione/5 Omelia.mp3'): '2013-0

In [9]:
from openai import OpenAI

with open("/Users/vigji/openai.txt") as f:
    api_key = f.read().strip()
client = OpenAI(api_key=api_key)

audio_file= open(test_omelia, "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)


In [11]:
transcript

Transcription(text="Spero che ci sia sempre dentro di me ancora un po' di stupore, dentro ognuno di noi, che prende parte dell'Eucaristia, perché le parole che abbiamo ascoltato a questi grani sono di una intensità turbante. La campagna elettorale mostra il volto più misero, degradato, di coloro che prima o poi verranno di nuovo o ancora a governarci, ed è la trasfigurazione negativa di ciò che anche il nostro Paese ha. E' la strutturazione della falsità in modo evidente, di promesse che nessuno sarà in grado mai di mantenere, di costruzioni e di sogni così da quattro soldi, nel quale nessuno riesce a ritrovare almeno un po' di sé. E sembra che coloro che proclamano tali verità siano come città che non si possono attraversare, città blindate, sembrano costruiti su colonne di ferro, sembrano muri di bronzo, e sembrano essere proprie le caratteristiche di quello che invece dovrebbe o è il profeta, il profeta colui che è chiamato fuori, che Dio addirittura conosce prima che si formasse ne

In [12]:
transcritp_filename = test_omelia.with_suffix(".txt")

with open(transcritp_filename, "w") as f:
    f.write(transcript.text)